In [1]:
import os
import json
import pandas as pd
import numpy as np
# Plot
import matplotlib.pyplot as plt
import seaborn as sns
# Warnings
import warnings
# Text Preprocessing and Natural Language Processing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import nltk
import re
import spacy
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
import missingno as msno
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive/BT4222/final_dataset


/content/drive/My Drive/BT4222/final_dataset


# Set up

In [4]:
jobs_df = pd.read_csv("final_jobs.csv")

In [5]:
courses_df = pd.read_csv("final_courses.csv")

In [6]:
courses_df.shape

(277491, 15)

In [7]:
jobs_df.shape

(1294346, 4)

In [8]:
courses_df

,reviews,reviewers,rating,name,institution,Overall Ratings,Level,Duration,Num of Reviews,skills,Instructor,description,Demeaned Rating,Fee,Popularity
0,"pretty dry, but i was able to pass with just t...",robert s,4,become a cbrs certified professional installer...,google - spectrum sharing,4.5,NaN,3,191,", communication, sas (software), regulations ...",google,This Specialization is intended for all learne...,-0.318181,599,859.5
1,would be a better experience if the video and ...,gabriel e r,4,become a cbrs certified professional installer...,google - spectrum sharing,4.5,NaN,3,191,", communication, sas (software), regulations ...",google,This Specialization is intended for all learne...,0.000001,599,859.5
2,information was perfect! the program itself wa...,jacob d,4,become a cbrs certified professional installer...,google - spectrum sharing,4.5,NaN,3,191,", communication, sas (software), regulations ...",google,This Specialization is intended for all learne...,-0.166666,599,859.5
3,a few grammatical mistakes on test made me do ...,dale b,4,become a cbrs certified professional installer...,google - spectrum sharing,4.5,NaN,3,191,", communication, sas (software), regulations ...",google,This Specialization is intended for all learne...,0.000001,599,859.5
4,excellent course and the training provided was...,sean g,4,become a cbrs certified professional installer...,google - spectrum sharing,4.5,NaN,3,191,", communication, sas (software), regulations ...",google,This Specialization is intended for all learne...,0.250001,599,859.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277486,great teacher!,george b s,5,technical support fundamentals,google,4.8,Beginner level,28,147,"binary code, customer support, linux, troubles...",google career certificates,This course is the first of a series that aims...,0.000001,49,705.6
277487,very satisfied,abubakarr k,5,technical support fundamentals,google,4.8,Beginner level,28,147,"binary code, customer support, linux, troubles...",google career certificates,This course is the first of a series that aims...,0.000001,49,705.6
277488,awesome course,md. i h,5,technical support fundamentals,google,4.8,Beginner level,28,147,"binary code, customer support, linux, troubles...",google career certificates,This course is the first of a series that aims...,0.125001,49,705.6
277489,comprehensive!,james k,5,technical support fundamentals,google,4.8,Beginner level,28,147,"binary code, customer support, linux, troubles...",google career certificates,This course is the first of a series that aims...,1.230770,49,705.6


In [9]:
jobs_df

,job_skills,job_title,search_position,job_level
0,"building custodial services, cleaning, janitor...",housekeeper i - pt,Cleaner,Mid senior
1,"customer service, restaurant management, food ...",assistant general manager - huntington 4131,Clerk General,Mid senior
2,"applied behavior analysis (aba), data analysis...",school-based behavior analyst,Consultant Education,Mid senior
3,"electrical engineering, project controls, sche...",electrical deputy engineering group supervisor,Cleaner,Mid senior
4,"electrical assembly, point to point wiring, st...",electrical assembly lead,Assembly Technician,Mid senior
...,...,...,...,...
1294341,"communication skills, time management, custome...",community ambassador - the station at raleigh,Public-Relations Representative,Mid senior
1294342,"windows sql, edi x12, edifecs platform, health...",sr. it analyst - edifecs technical systems ana...,Systems Analyst,Mid senior
1294343,"adaptability, communication, digital fluency, ...",operations excellence specialist,Cargo Checker,Associate
1294344,"cna, emt, bls, medical assistant, cpct, lpn, r...",float patient care associate cso ynhh,Medical Assistant,Mid senior


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
all_skills = pd.concat([jobs_df['job_skills'], courses_df['skills']], axis=0)

tfidf_matrix = vectorizer.fit_transform(all_skills)

In [11]:
from sklearn.cluster import KMeans

# Determine the optimal number of clusters (k) here, using methods like the Elbow method

num_clusters = 15  # Example number, adjust based on your analysis
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(tfidf_matrix)

jobs_df['Cluster'] = kmeans.predict(vectorizer.transform(jobs_df['job_skills']))
courses_df['Cluster'] = kmeans.predict(vectorizer.transform(courses_df['skills']))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [12]:
jobs_df

,job_skills,job_title,search_position,job_level,Cluster
0,"building custodial services, cleaning, janitor...",housekeeper i - pt,Cleaner,Mid senior,14
1,"customer service, restaurant management, food ...",assistant general manager - huntington 4131,Clerk General,Mid senior,13
2,"applied behavior analysis (aba), data analysis...",school-based behavior analyst,Consultant Education,Mid senior,7
3,"electrical engineering, project controls, sche...",electrical deputy engineering group supervisor,Cleaner,Mid senior,8
4,"electrical assembly, point to point wiring, st...",electrical assembly lead,Assembly Technician,Mid senior,14
...,...,...,...,...,...
1294341,"communication skills, time management, custome...",community ambassador - the station at raleigh,Public-Relations Representative,Mid senior,14
1294342,"windows sql, edi x12, edifecs platform, health...",sr. it analyst - edifecs technical systems ana...,Systems Analyst,Mid senior,7
1294343,"adaptability, communication, digital fluency, ...",operations excellence specialist,Cargo Checker,Associate,7
1294344,"cna, emt, bls, medical assistant, cpct, lpn, r...",float patient care associate cso ynhh,Medical Assistant,Mid senior,10


In [13]:
courses_df

,reviews,reviewers,rating,name,institution,Overall Ratings,Level,Duration,Num of Reviews,skills,Instructor,description,Demeaned Rating,Fee,Popularity,Cluster
0,"pretty dry, but i was able to pass with just t...",robert s,4,become a cbrs certified professional installer...,google - spectrum sharing,4.5,NaN,3,191,", communication, sas (software), regulations ...",google,This Specialization is intended for all learne...,-0.318181,599,859.5,8
1,would be a better experience if the video and ...,gabriel e r,4,become a cbrs certified professional installer...,google - spectrum sharing,4.5,NaN,3,191,", communication, sas (software), regulations ...",google,This Specialization is intended for all learne...,0.000001,599,859.5,8
2,information was perfect! the program itself wa...,jacob d,4,become a cbrs certified professional installer...,google - spectrum sharing,4.5,NaN,3,191,", communication, sas (software), regulations ...",google,This Specialization is intended for all learne...,-0.166666,599,859.5,8
3,a few grammatical mistakes on test made me do ...,dale b,4,become a cbrs certified professional installer...,google - spectrum sharing,4.5,NaN,3,191,", communication, sas (software), regulations ...",google,This Specialization is intended for all learne...,0.000001,599,859.5,8
4,excellent course and the training provided was...,sean g,4,become a cbrs certified professional installer...,google - spectrum sharing,4.5,NaN,3,191,", communication, sas (software), regulations ...",google,This Specialization is intended for all learne...,0.250001,599,859.5,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277486,great teacher!,george b s,5,technical support fundamentals,google,4.8,Beginner level,28,147,"binary code, customer support, linux, troubles...",google career certificates,This course is the first of a series that aims...,0.000001,49,705.6,5
277487,very satisfied,abubakarr k,5,technical support fundamentals,google,4.8,Beginner level,28,147,"binary code, customer support, linux, troubles...",google career certificates,This course is the first of a series that aims...,0.000001,49,705.6,5
277488,awesome course,md. i h,5,technical support fundamentals,google,4.8,Beginner level,28,147,"binary code, customer support, linux, troubles...",google career certificates,This course is the first of a series that aims...,0.125001,49,705.6,5
277489,comprehensive!,james k,5,technical support fundamentals,google,4.8,Beginner level,28,147,"binary code, customer support, linux, troubles...",google career certificates,This course is the first of a series that aims...,1.230770,49,705.6,5


# First interation

In [14]:
def recommend_courses(user_skills, jobs_df, courses_df, top_n=5):
    user_vec = vectorizer.transform([user_skills])
    user_cluster = kmeans.predict(user_vec)[0]

    # Filter jobs that match the user's skills cluster
    relevant_jobs = jobs_df[jobs_df['Cluster'] == user_cluster]['job_title'].unique()

    # Recommend courses that match job cluster
    recommended_courses = courses_df[courses_df['Cluster'] == user_cluster].sample(n=top_n)

    return relevant_jobs, recommended_courses[['name', 'institution', 'description']]

# Example usage
user_skills = "Python programming, data analysis, machine learning"
relevant_jobs, recommended_courses = recommend_courses(user_skills, jobs_df, courses_df)
print("Relevant Jobs:\n", relevant_jobs)
print("\nRecommended Courses:\n", recommended_courses)

Relevant Jobs:
 ['research scientist model engineer, autonomous startup' 'scientist i'
 'senior data analyst (product team) (bangkok based, relocation provided)'
 'application developer (autocad)' 'software developer senior'
 'senior field service technician' 'customer insight manager'
 'senior data engineer' 'senior sas programmer'
 'system architect/software developer' 'lead data analyst'
 'control engineer' 'sr. electrical controls engineer'
 'electrical / controls engineer - manufacturing'
 'automation engineer/controls engineer'
 'staff controls engineer, general assembly model 3'
 'data engineer 2 (rtp, nc hybrid role) #3328' 'biostatistician'
 'software engineer' 'industrial maintenance manager'
 'control systems programmer' 'senior manager, data science'
 'sr systems engineer'
 'marketing analyst (bangkok based, relocation provided)'
 'research and development engineer' 'firmware engineer'
 'software engineer - distributed systems' 'senior statistical programmer'
 'associate ma

In [15]:
user_skills = "communication, finance, presentation skills"
relevant_jobs, recommended_courses = recommend_courses(user_skills, jobs_df, courses_df)
print("Relevant Jobs:\n", relevant_jobs)
print("\nRecommended Courses:\n", recommended_courses)

Relevant Jobs:
 ['housekeeper i - pt' 'electrical assembly lead'
 'analyst, capital markets' ...
 'full-time store supervisor - brunswick square mall'
 'senior software engineer- asset & wealth management-new york-vice president'
 'community ambassador - the station at raleigh']

Recommended Courses:
                                                      name  \
199231  teamwork skills: communicating effectively in ...   
198891  teamwork skills: communicating effectively in ...   
198782  teamwork skills: communicating effectively in ...   
199083  teamwork skills: communicating effectively in ...   
199146  teamwork skills: communicating effectively in ...   

                           institution  \
199231  university of colorado boulder   
198891  university of colorado boulder   
198782  university of colorado boulder   
199083  university of colorado boulder   
199146  university of colorado boulder   

                                              description  
199231  Recognize

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Assume tfidf_matrix is your dataset from TF-IDF Vectorization
X = tfidf_matrix

# Elbow Method
wcss = []
for i in range(1, 16):  # Test k from 1 to 10
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10, 5))
plt.plot(range(1, 11), wcss, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# # Silhouette Method
# silhouette_coefficients = []
# for k in range(2, 11):  # Silhouette score cannot be computed for k=1
#     kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=0)
#     kmeans.fit(X)
#     score = silhouette_score(X, kmeans.labels_)
#     silhouette_coefficients.append(score)

# plt.figure(figsize=(10, 5))
# plt.plot(range(2, 11), silhouette_coefficients, marker='o')
# plt.title('Silhouette Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('Silhouette Coefficient')
# plt.show()


# Applying Domain Knowledge
Based on the Elbow Method graph, there isn't a clearly defined "elbow" which can sometimes be the case in practice. The curve is quite smooth and the decrease in WCSS is fairly consistent across the number of clusters.

We anticipate a broad range of distinct skills, therefore we opt for a larger number of clusters to capture these nuances.

 Given the diversity of job roles and industries represented in the dataset, we require more clusters to effectively separate distinct skill sets. For instance, the skill sets for 'Data Analysis' and 'Web Development' could be quite different, and we do not want them clustered together.

 Additionally, given the Course Content Complexity: Courses may range from very specialized to very general. Specialized courses (like "Advanced Neural Network Techniques") may need to be clustered separately from more general ones (like "Introduction to Programming").

 However, we also took into account the overlap of Skills: Some skills will be common across various jobs (e.g., "teamwork" or "communication"), so not every unique skill justifies a separate cluster. Similarly, while there may be many jobs, they might fall into several broad categories (e.g., IT, marketing, administration, etc.). Within these categories, the differentiation might not be as vast.

Therefore we have decided to set the number of clusters to be 10.

# Recommendations with Job Skills

In [26]:
user_skills_agg = courses_df.groupby('reviewers')['skills'].apply(lambda x: ', '.join(set(', '.join(x).split(', ')))).reset_index()

# Rename columns for clarity
user_skills_df = user_skills_agg.rename(columns={'reviewers': 'reviewer', 'skills': 'skills'})

# Display the first few rows of the new DataFrame
print(user_skills_df.head())

    reviewer                                             skills
0  #iam_kazi  code debugging ,, data structure, python progr...
1   #paris o  cognitive science, animal behavior, evolution,...
2       $ken  strategy and operations, risk management , sec...
3          (  , deep learning, machine learning, data scienc...
4        ( a  programming principles, python syntax and sema...


In [27]:
def select_random_user(courses_df):
    # Randomly select a user from the courses data
    random_user_index = random.randint(0, len(user_skills_df) - 1)
    random_user_skills = user_skills_df.iloc[random_user_index]['skills']
    return random_user_skills

In [28]:
import random

def recommend_courses(user_skills, jobs_df, courses_df, top_n=5):
    user_vec = vectorizer.transform([user_skills])
    user_cluster = kmeans.predict(user_vec)[0]

    # Filter jobs that match the user's skills cluster
    relevant_jobs = jobs_df[jobs_df['Cluster'] == user_cluster]['job_title'].unique()

    # Recommend courses that match job cluster
    recommended_courses = courses_df[courses_df['Cluster'] == user_cluster].sample(n=top_n)

    return relevant_jobs, recommended_courses[['name', 'institution', 'description']]

# Example usage
user_skills = select_random_user(courses_df)
relevant_jobs, recommended_courses = recommend_courses(user_skills, jobs_df, courses_df)
print("Relevant Jobs:\n", relevant_jobs)
print("\nRecommended Courses:\n", recommended_courses)

Relevant Jobs:
 ['school-based behavior analyst' 'senior lead technician/programmer'
 'program consultant' ...
 'senior principal engineer, application engineering'
 'longo toyota collision center estimator'
 'sr. it analyst - edifecs technical systems analyst - fully remote']

Recommended Courses:
                                                      name  \
181773     entrepreneurship 1: developing the opportunity   
215908         teach english now! foundational principles   
103477              fundamentals of quantitative modeling   
68038               excel skills for business: essentials   
198710  understanding clinical research: behind the st...   

                       institution  \
181773  university of pennsylvania   
215908    arizona state university   
103477  university of pennsylvania   
68038         macquarie university   
198710     university of cape town   

                                              description  
181773   \n Welcome to English for Career D

In [31]:
user_skills = select_random_user(courses_df)
relevant_jobs, recommended_courses = recommend_courses(user_skills, jobs_df, courses_df)
print("User's skills:\n", user_skills)
print("\nRelevant Jobs:\n", relevant_jobs)
print("\nRecommended Courses:\n", recommended_courses)

Relevant Jobs:
 programming principles, python syntax and semantics, computer programming, computational logic, data analysis, data structures, python programming , data structure, critical thinking, python programming, software engineering, data management, tuple, problem solving

Relevant Jobs:
 ['research scientist model engineer, autonomous startup' 'scientist i'
 'senior data analyst (product team) (bangkok based, relocation provided)'
 'application developer (autocad)' 'software developer senior'
 'senior field service technician' 'customer insight manager'
 'senior data engineer' 'senior sas programmer'
 'system architect/software developer' 'lead data analyst'
 'control engineer' 'sr. electrical controls engineer'
 'electrical / controls engineer - manufacturing'
 'automation engineer/controls engineer'
 'staff controls engineer, general assembly model 3'
 'data engineer 2 (rtp, nc hybrid role) #3328' 'biostatistician'
 'software engineer' 'industrial maintenance manager'
 'con

In [34]:
def recommend_relevant_courses(user_skills, jobs_df, courses_df, top_n=5):
    user_vec = vectorizer.transform([user_skills])
    user_cluster = kmeans.predict(user_vec)[0]

    # Filter jobs that match the user's skills cluster
    relevant_jobs = jobs_df[jobs_df['Cluster'] == user_cluster]['job_title'].unique()

    # We already have a vector for user_skills, which represents the user's current skill set
    user_skills_vec = user_vec  # Assuming user_skills is a string of skills the user has

    # Recommend courses that match job cluster but are not too similar to user's existing skills
    potential_courses = courses_df[courses_df['Cluster'] == user_cluster]

    # Calculate similarity of each course's skills to user's existing skills
    potential_courses_skills_vec = vectorizer.transform(potential_courses['skills'])
    similarity_scores = cosine_similarity(potential_courses_skills_vec, user_skills_vec).flatten()

    # Recommend courses that match job cluster
    recommended_courses = potential_courses[similarity_scores < 0.5].sample(n=top_n)

    return relevant_jobs, recommended_courses[['name', 'institution', 'description']]

# Example usage
user_skills = select_random_user(courses_df)
relevant_jobs, recommended_courses = recommend_relevant_courses(user_skills, jobs_df, courses_df)
print("User's skills:\n", user_skills)
print("Relevant Jobs:\n", relevant_jobs)
print("\nRecommended Courses:\n", recommended_courses)

User's skills:
 copy editing ,, short story writing, creativity, fiction writing
Relevant Jobs:
 ['marketing coordinator' 'senior account executive'
 'volunteer: reddit social media manager volunteer medicinal foods for stress, sleep and anxiety (crowddoing)'
 ... 'assistant director of marketing and recruitment'
 'social media manager, margaret mills' 'senior marketing professional']

Recommended Courses:
                                               name  \
114899              english for career development   
99772                    introduction to marketing   
114027              english for career development   
92506   management of fashion and luxury companies   
111105                          customer analytics   

                       institution  \
114899  university of pennsylvania   
99772   university of pennsylvania   
114027  university of pennsylvania   
92506            universit bocconi   
111105  university of pennsylvania   

                                   

Why <0.5?
Balance Between Relevance and Novelty: It aims to balance relevance (ensuring recommended courses are related to the user's interests and career goals) and novelty (ensuring the user learns something new). Too low of a threshold might recommend courses that are too dissimilar and possibly not useful, while too high might lead to recommendations that are too similar to what the user already knows.